In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, Normalizer
from xgboost import XGBClassifier
#import xgboost
from sklearn.metrics import accuracy_score

incidents = pd.read_csv("../sf-police-calls-for-service-and-incidents/label_encoded.csv")
print(incidents.head(10))

   Category  Time  Resolution  Day  Month  Friday  Monday  Saturday  Sunday  \
0        36   630          11   15      5       0       0         0       0   
1        20   254          11   15      5       0       0         0       0   
2        21   121           0   15      5       0       0         0       0   
3        21   121           0   15      5       0       0         0       0   
4        21   121           0   15      5       0       0         0       0   
5        21    87          11   15      5       0       0         0       0   
6        25    85           0   15      5       0       0         0       0   
7         7    85           0   15      5       0       0         0       0   
8        17    19           0   15      5       0       0         0       0   
9        37    19           0   15      5       0       0         0       0   

   Thursday     ...      BAYVIEW  CENTRAL  INGLESIDE  MISSION  NORTHERN  PARK  \
0         0     ...            0        0        

In [2]:
from sklearn.model_selection import train_test_split

incidents.pop('Resolution')
y = incidents.pop('Category')
y.head(10)

0    36
1    20
2    21
3    21
4    21
5    21
6    25
7     7
8    17
9    37
Name: Category, dtype: int64

In [3]:
X_train, X_test, y_train, y_test = train_test_split(incidents, y, test_size=0.2, random_state=42)

In [4]:
# Normalize
#normalizer = Normalizer()
#normalizer.fit_transform(X_train)

# Standardize
scaler = StandardScaler()
#X_train[['Time', 'Resolution']] = scaler.fit_transform(X_train[['Time', 'Resolution']])
X_train[['Time']] = scaler.fit_transform(X_train[['Time']])

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [5]:
X_train.head(10)

,Time,Day,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
489012,0.190183,15,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
821358,0.725054,6,12,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
809952,-1.338020,5,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1750574,0.113772,7,3,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
94026,0.546763,9,9,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2182620,0.304798,18,3,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
434483,0.877874,18,6,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
600710,1.119839,22,5,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
283043,-1.070585,18,6,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2069489,0.648643,20,12,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [9]:
params = {
    'booster': 'gbtree',
    'objective': 'multi:softprob',
    'verbosity': 0,
    'eta': .2,
    'min_child_weight': 4,
    'max_depth': 8,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'early_stopping_rounds': 5
}

xgb = XGBClassifier(**params)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, early_stopping_rounds=5, eta=0.2, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=8,
       min_child_weight=4, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.5, verbosity=0)

In [11]:
y_pred = xgb.predict(X_test)

In [12]:
accuracy_score(y_test, y_pred)

0.21757090777756458

In [14]:
pred_prob = xgb.predict_proba(X_test)

In [15]:
np.max(pred_prob)

0.40889132

In [ ]:
# 0.33665082786875994

In [16]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=31)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=31, verbose=0, warm_start=False)

In [17]:
accuracy_score(y_test, clf.predict(X_test))

0.23189128790871436

In [18]:
pred_prob = clf.predict_proba(X_test)

In [19]:
np.max(pred_prob)

0.36838658961872517